In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.optim as optim
import torchvision.utils as vutils

import helper

In [2]:
import wandb
wandb.init(project="gan-CIFAR10")

wandb: Currently logged in as: dankernel (use `wandb login --relogin` to force relogin)


In [3]:
wandb.config = {
  "learning_rate": 0.0002,
  "epochs": 200,
  "batch_size": 128
}

In [4]:
# ???
nc = 1

# input noise dimension
nz = 100
# number of generator filters
ngf = 64
# number of discriminator filters
ndf = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataloader

In [5]:
dataset = dset.MNIST(root="./data", download=True,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5), (0.5)),
                           ]))

In [6]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, num_workers=4)

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [8]:
netG = Generator().to(device)
netG.apply(helper.weights_init)
# netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)


In [10]:
netD = Discriminator().to(device)
netD.apply(helper.weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [11]:
criterion = nn.BCELoss() # Binary Cross entropy

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0005, betas=(0.5, 0.999))

In [12]:
fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1.0
fake_label = 0.0

In [ ]:
niter = 100
g_loss = []
d_loss = []

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch, niter, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        wandb.log({"lossG": errG.item()})
        wandb.log({"lossD": errD.item()})
        
        #save the output
        if i % 100 == 0:
            print('saving the output')
            vutils.save_image(real_cpu,'output/real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'output/fake_samples_epoch_%03d.png' % (epoch),normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'weights/netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'weights/netD_epoch_%d.pth' % (epoch))


[0/100][0/469] Loss_D: 1.8517 Loss_G: 2.5409 D(x): 0.3016 D(G(z)): 0.3340 / 0.1009
saving the output
[0/100][1/469] Loss_D: 11.1446 Loss_G: 7.0636 D(x): 1.0000 D(G(z)): 1.0000 / 0.0017
[0/100][2/469] Loss_D: 4.5642 Loss_G: 16.5672 D(x): 0.9968 D(G(z)): 0.9791 / 0.0000
[0/100][3/469] Loss_D: 1.1133 Loss_G: 7.6293 D(x): 0.5784 D(G(z)): 0.0240 / 0.0010
[0/100][4/469] Loss_D: 6.6913 Loss_G: 17.2337 D(x): 0.9750 D(G(z)): 0.9975 / 0.0000
[0/100][5/469] Loss_D: 0.1724 Loss_G: 14.9365 D(x): 0.9237 D(G(z)): 0.0042 / 0.0000
[0/100][6/469] Loss_D: 0.6863 Loss_G: 17.6463 D(x): 0.9075 D(G(z)): 0.3343 / 0.0000
[0/100][7/469] Loss_D: 0.1584 Loss_G: 11.3062 D(x): 0.9201 D(G(z)): 0.0121 / 0.0000
[0/100][8/469] Loss_D: 3.4698 Loss_G: 29.8834 D(x): 0.9546 D(G(z)): 0.9504 / 0.0000
[0/100][9/469] Loss_D: 2.2009 Loss_G: 32.8550 D(x): 0.3456 D(G(z)): 0.0000 / 0.0000
[0/100][10/469] Loss_D: 0.0135 Loss_G: 32.9133 D(x): 0.9883 D(G(z)): 0.0000 / 0.0000
[0/100][11/469] Loss_D: 0.0156 Loss_G: 31.6286 D(x): 0.9902

[0/100][97/469] Loss_D: 0.4079 Loss_G: 4.1579 D(x): 0.8571 D(G(z)): 0.1738 / 0.0281
[0/100][98/469] Loss_D: 0.4445 Loss_G: 3.6097 D(x): 0.8291 D(G(z)): 0.1537 / 0.0407
[0/100][99/469] Loss_D: 0.6160 Loss_G: 4.8620 D(x): 0.8520 D(G(z)): 0.2930 / 0.0143
[0/100][100/469] Loss_D: 0.7293 Loss_G: 3.4728 D(x): 0.7130 D(G(z)): 0.1938 / 0.0611
saving the output
[0/100][101/469] Loss_D: 0.7170 Loss_G: 6.6995 D(x): 0.8647 D(G(z)): 0.3513 / 0.0028
[0/100][102/469] Loss_D: 1.1140 Loss_G: 1.4144 D(x): 0.5013 D(G(z)): 0.0363 / 0.3483
[0/100][103/469] Loss_D: 1.9827 Loss_G: 10.4510 D(x): 0.9728 D(G(z)): 0.7630 / 0.0006
[0/100][104/469] Loss_D: 1.1725 Loss_G: 5.8017 D(x): 0.4855 D(G(z)): 0.0044 / 0.0176
[0/100][105/469] Loss_D: 0.3202 Loss_G: 3.1915 D(x): 0.9039 D(G(z)): 0.1370 / 0.0798
[0/100][106/469] Loss_D: 1.1227 Loss_G: 7.6253 D(x): 0.8806 D(G(z)): 0.5049 / 0.0010
[0/100][107/469] Loss_D: 3.2966 Loss_G: 0.5698 D(x): 0.0925 D(G(z)): 0.0053 / 0.6697
[0/100][108/469] Loss_D: 2.6237 Loss_G: 17.6949 D

[0/100][194/469] Loss_D: 0.7674 Loss_G: 4.8356 D(x): 0.8811 D(G(z)): 0.4256 / 0.0140
[0/100][195/469] Loss_D: 0.8510 Loss_G: 1.4200 D(x): 0.5161 D(G(z)): 0.0433 / 0.3015
[0/100][196/469] Loss_D: 0.9191 Loss_G: 6.4250 D(x): 0.9674 D(G(z)): 0.5266 / 0.0030
[0/100][197/469] Loss_D: 0.7010 Loss_G: 2.8665 D(x): 0.5618 D(G(z)): 0.0183 / 0.0848
[0/100][198/469] Loss_D: 0.4486 Loss_G: 4.0309 D(x): 0.9586 D(G(z)): 0.3023 / 0.0295
[0/100][199/469] Loss_D: 0.4005 Loss_G: 3.6990 D(x): 0.8415 D(G(z)): 0.1680 / 0.0394
[0/100][200/469] Loss_D: 0.4503 Loss_G: 2.2919 D(x): 0.7724 D(G(z)): 0.1329 / 0.1404
saving the output
[0/100][201/469] Loss_D: 0.5460 Loss_G: 4.2483 D(x): 0.9001 D(G(z)): 0.3170 / 0.0191
[0/100][202/469] Loss_D: 0.5485 Loss_G: 1.9078 D(x): 0.6676 D(G(z)): 0.0499 / 0.1846
[0/100][203/469] Loss_D: 0.5221 Loss_G: 4.0524 D(x): 0.9258 D(G(z)): 0.3285 / 0.0249
[0/100][204/469] Loss_D: 0.3788 Loss_G: 2.7526 D(x): 0.7691 D(G(z)): 0.0713 / 0.0839
[0/100][205/469] Loss_D: 0.3257 Loss_G: 3.1235 

[0/100][291/469] Loss_D: 0.7205 Loss_G: 1.5167 D(x): 0.6804 D(G(z)): 0.2311 / 0.2584
[0/100][292/469] Loss_D: 0.8474 Loss_G: 1.8572 D(x): 0.7193 D(G(z)): 0.3507 / 0.1950
[0/100][293/469] Loss_D: 0.7997 Loss_G: 1.9364 D(x): 0.7080 D(G(z)): 0.3125 / 0.1822
[0/100][294/469] Loss_D: 0.6775 Loss_G: 2.0934 D(x): 0.7554 D(G(z)): 0.2801 / 0.1548
[0/100][295/469] Loss_D: 0.6830 Loss_G: 1.7426 D(x): 0.7124 D(G(z)): 0.2510 / 0.2064
[0/100][296/469] Loss_D: 0.6311 Loss_G: 2.0203 D(x): 0.7608 D(G(z)): 0.2636 / 0.1618
[0/100][297/469] Loss_D: 0.5066 Loss_G: 2.0842 D(x): 0.7806 D(G(z)): 0.2026 / 0.1525
[0/100][298/469] Loss_D: 0.5617 Loss_G: 2.2084 D(x): 0.7950 D(G(z)): 0.2529 / 0.1299
[0/100][299/469] Loss_D: 0.5555 Loss_G: 2.0548 D(x): 0.7606 D(G(z)): 0.2134 / 0.1550
[0/100][300/469] Loss_D: 0.5124 Loss_G: 1.9211 D(x): 0.7838 D(G(z)): 0.2102 / 0.1753
saving the output
[0/100][301/469] Loss_D: 0.5755 Loss_G: 2.1308 D(x): 0.7869 D(G(z)): 0.2539 / 0.1423
[0/100][302/469] Loss_D: 0.5448 Loss_G: 1.9389 

[0/100][388/469] Loss_D: 1.2347 Loss_G: 4.2519 D(x): 0.9491 D(G(z)): 0.6445 / 0.0202
[0/100][389/469] Loss_D: 2.3484 Loss_G: 0.3125 D(x): 0.1252 D(G(z)): 0.0288 / 0.7531
[0/100][390/469] Loss_D: 1.8426 Loss_G: 2.4169 D(x): 0.9553 D(G(z)): 0.7795 / 0.1162
[0/100][391/469] Loss_D: 0.8765 Loss_G: 2.0755 D(x): 0.5784 D(G(z)): 0.2023 / 0.1542
[0/100][392/469] Loss_D: 0.9620 Loss_G: 0.7612 D(x): 0.5568 D(G(z)): 0.2438 / 0.5004
[0/100][393/469] Loss_D: 1.0087 Loss_G: 2.6349 D(x): 0.8424 D(G(z)): 0.5267 / 0.0965
[0/100][394/469] Loss_D: 1.0680 Loss_G: 0.8645 D(x): 0.4664 D(G(z)): 0.1720 / 0.4625
[0/100][395/469] Loss_D: 1.0004 Loss_G: 2.5421 D(x): 0.8094 D(G(z)): 0.4998 / 0.1024
[0/100][396/469] Loss_D: 1.0104 Loss_G: 0.7837 D(x): 0.4760 D(G(z)): 0.1403 / 0.4961
[0/100][397/469] Loss_D: 1.1482 Loss_G: 2.8200 D(x): 0.8676 D(G(z)): 0.5824 / 0.0796
[0/100][398/469] Loss_D: 1.0836 Loss_G: 0.9498 D(x): 0.4504 D(G(z)): 0.1314 / 0.4476
[0/100][399/469] Loss_D: 0.9809 Loss_G: 2.3886 D(x): 0.8396 D(G(z

...

[25/100][276/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][277/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][278/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][279/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][280/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][281/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][282/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][283/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][284/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][285/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][286/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][287/469] Loss_D: 100.00

[25/100][370/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][371/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][372/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][373/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][374/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][375/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][376/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][377/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][378/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][379/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][380/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][381/469] Loss_D: 100.00

[25/100][464/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][465/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][466/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][467/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[25/100][468/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][0/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
saving the output
[26/100][1/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][2/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][3/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][4/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][5/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][6/469] Loss_D: 10

[26/100][91/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][92/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][93/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][94/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][95/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][96/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][97/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][98/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][99/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][100/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
saving the output
[26/100][101/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][102/469] Loss_

[26/100][185/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][186/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][187/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][188/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][189/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][190/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][191/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][192/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][193/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][194/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][195/469] Loss_D: 100.0000 Loss_G: 0.0000 D(x): 1.0000 D(G(z)): 1.0000 / 1.0000
[26/100][196/469] Loss_D: 100.00

In [ ]:
wandb.watch(netG)

# Ref
- https://github.com/Ksuryateja/DCGAN-CIFAR10-pytorch/blob/master/gan_cifar.py

In [ ]:
batch_size = 5
print(batch_size)
noise = torch.randn(batch_size, nz, 1, 1, device=device)
print("noise", noise.shape)
fake = netG(noise)
print(fake.shape)
vutils.save_image(fake.detach(),'debug.png',normalize=True)